In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

from ucnpexp.instruments import OscilloscopeChannel, Spectrometer
import numpy as np
import rpyc

In [3]:

conn = rpyc.connect('rp-f05512.local', port=18861)

In [4]:

osc = OscilloscopeChannel(conn)

In [5]:

window = 0
offset = 0
osc.set_measurement_time(3, offset=offset)
datapoints_for_time = osc.amount_datapoints
buff_offset = window * osc.amount_datapoints
osc.set_trigger(channel=1, edge="neg")
print(f"{osc.decimation=}\n{osc.trigger_post=}\n{osc.trigger_pre=}\n{osc.trig_src=}\n{osc.get_measurement_time()=}\n{osc.amount_datapoints=}\n{osc.sampling_rate=}")

osc.decimation=32768
osc.trigger_post=11444
osc.trigger_pre=0
osc.trig_src=8
osc.get_measurement_time()=2.999975936
osc.amount_datapoints=11444


In [6]:
def acquire_decay(lifetime, amount_windows=1, amount_buffers=1, 
                  pct_overlap=0, measurement_length=9.0,
                  path=None, threshold=2):
    counts = np.array([]) 
    for window in range(amount_windows):
        osc.set_measurement_time(lifetime)
        offset = (1 - pct_overlap)*window*osc.get_measurement_time()
        if offset + osc.get_measurement_time() > measurement_length:
            print("Offset and measurement time exceed simulated length")
            print("returning...")
            return
        osc.set_measurement_time(lifetime, offset=offset)
        time_axis = np.arange(0, osc.amount_datapoints, 1.0)/osc.sampling_rate + offset
        if path is not None:
            np.save(f"{path}/times_w{window}", time_axis)
        for buffer in range(amount_buffers):
            print(f"asking for window {window} and buffer {buffer}")
            screen = np.array(osc.get_triggered())
            if path is not None:
                np.save(
                f"{path}/w{window}b{buffer}ol{pct_overlap}l{lifetime}",
                  screen
                  )
            peaks = np.where(np.diff(screen) > threshold)[0]
            times = peaks/osc.sampling_rate + offset
            counts = np.hstack((counts, times))
    return counts

In [10]:
lifetime = 3.0
amount_windows = 6
amount_buffers = 100
pct_overlap = 0.7
path = '/home/tomi/Documents/facultad/tesis/horibaPTI-controller/calibration/rp_decay/data/2023-12-13/n6-p10-t10-ol0.3/'
counts = acquire_decay(lifetime,
                       amount_windows=amount_windows,
                       amount_buffers=amount_buffers,
                       pct_overlap=pct_overlap,
                       path=path
                       )

asking for window 0 and buffer 0
asking for window 0 and buffer 1
asking for window 0 and buffer 2
asking for window 0 and buffer 3
asking for window 0 and buffer 4
asking for window 0 and buffer 5
asking for window 0 and buffer 6
asking for window 0 and buffer 7
asking for window 0 and buffer 8
asking for window 0 and buffer 9
asking for window 0 and buffer 10
asking for window 0 and buffer 11
asking for window 0 and buffer 12
asking for window 0 and buffer 13
asking for window 0 and buffer 14
asking for window 0 and buffer 15
asking for window 0 and buffer 16
asking for window 0 and buffer 17
asking for window 0 and buffer 18
asking for window 0 and buffer 19
asking for window 0 and buffer 20
asking for window 0 and buffer 21
asking for window 0 and buffer 22
asking for window 0 and buffer 23
asking for window 0 and buffer 24
asking for window 0 and buffer 25
asking for window 0 and buffer 26
asking for window 0 and buffer 27
asking for window 0 and buffer 28
asking for window 0 and 